<a href="https://colab.research.google.com/github/uhrelee/sephora_nlp/blob/main/sephora_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sephora Products and Skincare Reviews

I am using a dataset from Kaggle that includes reviews on skincare products featured on Sephora's website. The dataset includes multiple CSV files, primarily a csv that describes over 8,000 beauty products (product and brand names, ingredients, reviews, etc.) and a csv on the reviews on those products.

In [1]:
!pip install kaggle

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls "/content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews.zip"

'/content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews.zip'


In [4]:
!unzip "/content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews.zip" -d "/content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews"

Archive:  /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews.zip
replace /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/product_info.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/product_info.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/product_info.csv  
  inflating: /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/reviews_0-250.csv  
  inflating: /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/reviews_1250-end.csv  
  inflating: /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/reviews_250-500.csv  
  inflating: /content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/reviews_500-750.csv  
  inflating: /content/drive/My Drive/sephora_nlp/sephora-produ

In [5]:
import pandas as pd

base_path = '/content/drive/My Drive/sephora_nlp/sephora-products-and-skincare-reviews/'

products = pd.read_csv(base_path + 'product_info.csv')
r250 = pd.read_csv(base_path + 'reviews_0-250.csv')
r500 = pd.read_csv(base_path + 'reviews_250-500.csv')
r750 = pd.read_csv(base_path + 'reviews_500-750.csv')
r1250 = pd.read_csv(base_path + 'reviews_750-1250.csv')
rEnd = pd.read_csv(base_path + 'reviews_1250-end.csv')

reviews = pd.concat([r250, r500, r750, r1250, rEnd])
orig_reviews_len = len(reviews)
print(orig_reviews_len)

/tmp/ipython-input-3846886663.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  r250 = pd.read_csv(base_path + 'reviews_0-250.csv')
/tmp/ipython-input-3846886663.py:9: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  r1250 = pd.read_csv(base_path + 'reviews_750-1250.csv')
/tmp/ipython-input-3846886663.py:10: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  rEnd = pd.read_csv(base_path + 'reviews_1250-end.csv')


1094411


# Data Cleaning and EDA
Time to do some data cleaning!

First, I have to make sure that the 'author_id' columns in the review tables are clean of any dummy variables. In this case, the removal of these values won't skew my data since they are placeholders.

In [6]:
# Convert 'author_id' to floats, any errors become NaN
reviews['author_id'] = pd.to_numeric(reviews['author_id'], errors='coerce')

# Drop NaN (dummy values)
reviews.dropna(subset=['author_id'], inplace=True)

# Change to int type
reviews['author_id'] = reviews['author_id'].astype(int)

print("Data types after cleaning 'author_id':")
print(reviews['author_id'].apply(type).value_counts())
print(f"Rows removed: {orig_reviews_len - len(reviews)}")

Data types after cleaning 'author_id':
author_id
<class 'int'>    1094347
Name: count, dtype: int64
Rows removed: 64


Now I would like to check if every review has a valid product_id.

Alternatively, I'd like to know if every product has a review.

In [11]:
invalid_reviews = reviews[~reviews['product_id'].isin(products['product_id'])]
len(invalid_reviews)
# goal is 0

0

In [10]:
products_without_reviews = products[~products['product_id'].isin(reviews['product_id'])]
len(products_without_reviews)
# There are 6143 products without any reviews.

6143